# Análisis y Recategorización de Productos - Dataset Aurelion

Este notebook analiza los productos de Aurelion para identificar y corregir productos mal categorizados.

## Objetivos
1. Importar librerías y configuración
2. Cargar archivo productos.xlsx
3. Definir funciones de categorización
4. Generar sugerencias y actualizar archivo
5. Resumen y revisión de resultados

## Paso 1: Importar librerías y configuración

Se importan las librerías necesarias para el análisis y se configura la ruta de trabajo.

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import os
from pathlib import Path

# Configurar ruta de trabajo
ruta_aurelion = Path(r'c:\Users\WIGUSA\Documents\Github\AI-Fundamentals-Guayerd-IBM\AurelionProject\data')

## Paso 2: Cargar archivo productos.xlsx

Se carga el archivo de productos para su exploración inicial.

In [ ]:
# Cargar el archivo de productos
productos_df = pd.read_excel(ruta_aurelion / 'productos.xlsx')

# Mostrar las primeras filas
productos_df.head()

## Paso 3: Definir funciones de categorización

Se definen los criterios y funciones para categorizar los productos según palabras clave.

In [ ]:
# Definir diccionario de palabras clave por categoría
keywords_mejorado = {
    'Alimentos': {
        'bebidas': ['coca', 'pepsi', 'sprite', 'fanta', 'agua', 'jugo', 'gaseosa', 'soda', 'cerveza', 'vino', 'refresco', 'bebida', 'energética', 'energetica', 'bebidas'],
        'lacteos': ['leche', 'yogur', 'yoghurt', 'queso', 'crema', 'manteca', 'dulce de leche', 'postrecito'],
        'panificados': ['pan', 'galletas', 'galletitas', 'torta', 'bizcocho', 'facturas', 'alfajor', 'tostadas', 'pan rallado'],
        'cereales_granos': ['arroz', 'fideos', 'pasta', 'cereal', 'avena', 'harina', 'garbanzos', 'lentejas', 'porotos', 'arvejas', 'maíz', 'maiz'],
        'snacks_golosinas': ['papas', 'chips', 'palitos', 'maní', 'nueces', 'mix', 'caramelo', 'chicle', 'chupetín', 'chupetin', 'turrón', 'turron', 'gomitas', 'chocolate', 'dulces', 'golosinas'],
        'comidas_preparadas': ['pizza', 'pizzas', 'hamburguesa', 'hamburguesas', 'burger', 'hot dog', 'hotdog', 'empanada', 'empanadas', 'arepa', 'sushi', 'congelada', 'congeladas'],
        'conservas_salsas': ['atún', 'sardinas', 'tomate', 'salsa', 'pure', 'mermelada', 'conserva', 'enlatado', 'pickles', 'verdura', 'verduras'],
        'condimentos': ['sal', 'pimienta', 'aceite', 'vinagre', 'mayonesa', 'mostaza', 'ketchup', 'condimento', 'especias'],
        'infusiones': ['café', 'cafe', 'te', 'té', 'mate', 'yerba', 'saquitos', 'molido', 'cacao', 'manzanilla', 'tilo']
    },
    'Limpieza': {
        'limpieza_hogar': ['detergente', 'jabón', 'jabon', 'lavandina', 'desinfectante', 'limpiador', 'limpia', 'lustra', 'quita', 'cera', 'perfumina', 'suavizante', 'quitamanchas', 'antigrasa', 'limpiavidrios', 'desengrasante'],
        'utensilios_limpieza': ['escoba', 'trapo', 'esponja', 'cepillo', 'balde', 'pala', 'paño', 'secador', 'plumero', 'guantes'],
        'papel_descartables': ['papel', 'servilletas', 'rollo', 'pañuelos', 'tissue', 'toallas', 'absorbente', 'cocina'],
        'bolsas': ['bolsa', 'bolsas', 'residuos', 'consorcio', 'freezer'],
        'higiene_personal': ['shampoo', 'acondicionador', 'pasta', 'dental', 'cepillo', 'dientes', 'desodorante', 'toallitas', 'algodón', 'algodon', 'hilo dental', 'enjuague', 'bucal', 'perfume', 'crema dental', 'mascarilla', 'capilar']
    }
}

# Función para detectar categoría
def detectar_categoria_mejorada(nombre_producto):
    nombre = str(nombre_producto).lower()
    palabras_clave_alimentos = []
    for subcat in keywords_mejorado['Alimentos'].values():
        palabras_clave_alimentos.extend(subcat)
    palabras_clave_limpieza = []
    for subcat in keywords_mejorado['Limpieza'].values():
        palabras_clave_limpieza.extend(subcat)
    tiene_alimentos = any(palabra in nombre for palabra in palabras_clave_alimentos)
    tiene_limpieza = any(palabra in nombre for palabra in palabras_clave_limpieza)
    if tiene_alimentos and tiene_limpieza:
        return 'Limpieza'
    if tiene_limpieza:
        return 'Limpieza'
    if tiene_alimentos:
        return 'Alimentos'
    patrones_limpieza = ['limpia', 'limpio', 'higien', 'desinfect', 'sanitiz', 'antibac', 'anti-bac', 'aseo', 'baño', 'lavado']
    if any(patron in nombre for patron in patrones_limpieza):
        return 'Limpieza'
    return 'Alimentos'

## Paso 4: Generar sugerencias y actualizar archivo

Se generan las sugerencias de categorización y se actualiza el archivo principal con las nuevas categorías.

In [ ]:
# Función para generar sugerencias
def generar_sugerencias_categorizacion_mejorada():
    sugerencias = []
    for idx, row in productos_df.iterrows():
        categoria_sugerida = detectar_categoria_mejorada(row.get('nombre_producto', ''))
        sugerencias.append({
            'id_producto': row.get('id_producto'),
            'nombre_producto': row.get('nombre_producto'),
            'categoria_actual': row.get('categoria'),
            'categoria_sugerida': categoria_sugerida,
            'precio_unitario': row.get('precio_unitario'),
            'requiere_revision': 'Sí' if categoria_sugerida != row.get('categoria') else 'No'
        })
    return pd.DataFrame(sugerencias)

print("=" * 80)
print("INICIO DE LA EJECUCIÓN")
print("=" * 80)

# 1. Generar sugerencias
df_sugerencias_mejoradas = generar_sugerencias_categorizacion_mejorada()

# 2. Actualizar el archivo productos.xlsx con las categorías corregidas
productos_df['categoria'] = productos_df['nombre_producto'].apply(detectar_categoria_mejorada)
productos_df.to_excel(ruta_aurelion / 'productos.xlsx', index=False)
print(f"\nArchivo productos.xlsx actualizado con las categorías corregidas.")

# 3. Exportar sugerencias a archivo temporal
output_dir = ruta_aurelion / 'output'
output_dir.mkdir(exist_ok=True)
fecha_actual = pd.Timestamp.now().strftime('%Y%m%d_%H%M')
archivo_sugerencias = output_dir / f'sugerencias_categorizacion_mejorada_{fecha_actual}.xlsx'
df_sugerencias_mejoradas.to_excel(archivo_sugerencias, index=False)
print(f"\nArchivo temporal de sugerencias generado: {archivo_sugerencias}")

# 4. Eliminar archivo temporal
try:
    os.remove(archivo_sugerencias)
    print(f"\nArchivo temporal eliminado: {archivo_sugerencias}")
except Exception as e:
    print(f"No se pudo eliminar el archivo temporal: {e}")

print("\n" + "=" * 80)
print("FIN DE LA EJECUCIÓN")
print("=" * 80)

## Paso 5: Resumen y revisión de resultados

Se muestra un resumen de las sugerencias generadas y los productos que requieren revisión.

In [ ]:
# Mostrar resumen de cambios
print("\nResumen de nuevas sugerencias:")
print("-" * 50)
print(f"Total de productos: {len(df_sugerencias_mejoradas)}")
print(f"Productos que requieren revisión: {len(df_sugerencias_mejoradas[df_sugerencias_mejoradas['requiere_revision'] == 'Sí'])}")

print("\nDesglose de sugerencias por categoría sugerida:")
print("-" * 50)
print(df_sugerencias_mejoradas['categoria_sugerida'].value_counts())

# Mostrar productos que requieren cambio de categoría
productos_cambio = df_sugerencias_mejoradas[df_sugerencias_mejoradas['requiere_revision'] == 'Sí']
if len(productos_cambio) > 0:
    print("\nProductos que requieren cambio de categoría (muestra):")
    print("-" * 50)
    print(productos_cambio[['id_producto', 'nombre_producto', 'categoria_actual', 'categoria_sugerida']].head(20))